In [70]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

from collections import defaultdict

In [71]:
# Define the path
#path = os.path.join("GitHub", "DataProcessing")

# Print the path
#print(path)

In [72]:
#pwd

In [73]:
#td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [74]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

In [75]:
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']

In [76]:
# Create dictionaries to store the counts and most mentioned names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)
mostMentionedNamesDict = {}

# Function to find participant data in kill events
def findParticipantData(data):
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    
                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

# Example usage
findParticipantData(data)

# Process the counts and choose the most mentioned champion name for each ID
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    mostMentionedNamesDict[participantId] = mostMentionedChampion
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")

# Access the most mentioned champion name for a specific participant ID
# Example: print(mostMentionedNamesDict[1])
mostMentionedNamesDict


{1: 'Volibear',
 6: 'Sion',
 9: 'Jhin',
 5: 'Brand',
 4: 'Kaisa',
 3: 'Hwei',
 8: 'Veigar',
 7: 'Kayn',
 2: 'Briar',
 10: 'Nautilus'}

In [77]:
participantData = []

participantChampionMapping = {
}

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)

findParticipantData(data)

participantData[0]


[1,
 'NLZhdGCfh-EgdvaODKW2IlCFzJBMtE4Dr9SFgKNpE1UC6GDNH2u2TQmGZoqSLXcVjo-erCaRQVV0cQ',
 'Volibear']

In [78]:
# Create a dictionary to store the counts of champion names for each ID
championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

# Function to find participant data in kill events
def findParticipantDataInKillEvents(data):
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    
                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)

# Example usage
findParticipantDataInKillEvents(data)

# Process the counts and choose the most mentioned champion name for each ID
for participantId, championCounts in championCountsDict.items():
    mostMentionedChampion = max(championCounts, key=championCounts.get)
    count = championCounts[mostMentionedChampion]
    names = namesDict[participantId]

    # print(f"Participant ID: {participantId}, Most Mentioned Champion: {mostMentionedChampion} (Count: {count})")
    # print(f"Names associated with ID {participantId}: {', '.join(names)}\n")


In [79]:
# Get every timestamp within a game paired with corresponding minute
def getTimestamps():
    for counter, frame in enumerate(data['info']['frames']):
        timestamp = frame['timestamp']
        print(f"{counter + 1} Timestamp = {timestamp}")

getTimestamps()

1 Timestamp = 0
2 Timestamp = 60016
3 Timestamp = 120021
4 Timestamp = 180035
5 Timestamp = 240035
6 Timestamp = 300066
7 Timestamp = 360069
8 Timestamp = 420084
9 Timestamp = 480090
10 Timestamp = 540109
11 Timestamp = 600126
12 Timestamp = 660130
13 Timestamp = 720141
14 Timestamp = 780169
15 Timestamp = 840170
16 Timestamp = 900173
17 Timestamp = 960186
18 Timestamp = 1020272
19 Timestamp = 1080286
20 Timestamp = 1140318
21 Timestamp = 1200328
22 Timestamp = 1260354
23 Timestamp = 1320366
24 Timestamp = 1380386
25 Timestamp = 1440411
26 Timestamp = 1500414
27 Timestamp = 1560433
28 Timestamp = 1620447
29 Timestamp = 1680454
30 Timestamp = 1740480
31 Timestamp = 1800496
32 Timestamp = 1860523
33 Timestamp = 1892560


In [80]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData

In [81]:
def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key}:")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

In [82]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = "2"
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120021
championStats:
    armor: 33
    attackDamage: 67
    attackSpeed: 114
    health: 520
    healthMax: 684
    healthRegen: 7
    magicResist: 33
    movementSpeed: 340
  currentGold: 199
damageStats:
    totalDamageDone: 3801
    totalDamageTaken: 709
  jungleMinionsKilled: 5
  level: 2
  participantId: 2
position:
    x: 8451
    y: 2647
  totalGold: 649
  xp: 367

---



In [83]:
def getChampionKilled(data, startTimestamp, endTimestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        if startTimestamp <= timestamp <= endTimestamp:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killTimestamp = event['timestamp']
                    print(f"\nTimestamp of CHAMPION_KILL event: {killTimestamp}")

                    # Get the champion name of the killed participant using victimId
                    victimId = event.get('victimId')
                    victimData = next((p for p in participantData if p[0] == victimId), None)
                    if victimData:
                        victimChampionName = victimData[2]
                        print(f"Champion killed: {victimChampionName} (ID: {victimId})")

                        print("Victim Damage Received:")
                        processedNames = set()
                        for damageInfo in event.get('victimDamageReceived', []):
                            name = damageInfo['name']
                            if name in [p[2] for p in participantData] and name not in processedNames:
                                print(f"  Name: {name}")
                                processedNames.add(name)
                    else:
                        print(f"No participant data found for victim ID: {victimId}\n")

# Example usage: Specify the start and end timestamps
startTimestamp = 120000  # Replace with your desired start timestamp
endTimestamp = 200000    # Replace with your desired end timestamp

# Call the function with the specified time frame
getChampionKilled(data, startTimestamp, endTimestamp)



Timestamp of CHAMPION_KILL event: 134226
Champion killed: Sion (ID: 6)
Victim Damage Received:
  Name: Volibear

Timestamp of CHAMPION_KILL event: 171327
Champion killed: Brand (ID: 5)
Victim Damage Received:
  Name: Nautilus
  Name: Jhin


In [84]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def getEventOfChampionKillAtTimestamp(data, targetTimestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == targetTimestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
targetTimestamp = 134226
championKillEvent = getEventOfChampionKillAtTimestamp(data, targetTimestamp)

# Print or use the champion_kill_event data as needed
if championKillEvent:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(championKillEvent)
else:
    print(f"No CHAMPION_KILL event found at timestamp {targetTimestamp}.")


CHAMPION_KILL event data:
{'bounty': 400,
 'killStreakLength': 0,
 'killerId': 1,
 'position': {'x': 3958, 'y': 13447},
 'shutdownBounty': 0,
 'timestamp': 134226,
 'type': 'CHAMPION_KILL',
 'victimDamageDealt': [{'basic': False,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 233,
                        'spellName': 'sionq',
                        'spellSlot': 0,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': True,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 105,
                        'spellName': 'sionbasicattack3',
                        'spellSlot': 66,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': Fa

In [85]:
# Get timestamp at every kill throughout the match
def getTimestampAtKill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')

                # Get the champion name of the killed participant using victimId
                victimData = next((p for p in participantData if p[0] == victimId), None)
                if victimData:
                    victimChampionName = victimData[2]

                    # Get the killer's participant data
                    killerData = next((p for p in participantData if p[0] == killerId), None)
                    if killerData:
                        killerName = killerData[2]
                        print(f"{victimChampionName} was killed with ID: {victimId}")
                        print(f"Timestamp of CHAMPION_KILL: {timestamp}")
                        print(f"Killed by {killerName} with ID: {killerId}\n")
                    else:
                        print(f"No participant data found for killer ID: {killerId}\n")
                else:
                    print(f"No participant data found for victim ID: {victimId}\n")

getTimestampAtKill(data)


Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 134226
Killed by Volibear with ID: 1

Brand was killed with ID: 5
Timestamp of CHAMPION_KILL: 171327
Killed by Jhin with ID: 9

Kaisa was killed with ID: 4
Timestamp of CHAMPION_KILL: 187450
Killed by Jhin with ID: 9

Veigar was killed with ID: 8
Timestamp of CHAMPION_KILL: 192190
Killed by Hwei with ID: 3

Hwei was killed with ID: 3
Timestamp of CHAMPION_KILL: 209499
Killed by Kayn with ID: 7

Kayn was killed with ID: 7
Timestamp of CHAMPION_KILL: 255814
Killed by Briar with ID: 2

Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 260683
Killed by Volibear with ID: 1

Nautilus was killed with ID: 10
Timestamp of CHAMPION_KILL: 303299
Killed by Kaisa with ID: 4

Kayn was killed with ID: 7
Timestamp of CHAMPION_KILL: 352366
Killed by Briar with ID: 2

Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 389728
Killed by Volibear with ID: 1

Veigar was killed with ID: 8
Timestamp of CHAMPION_KILL: 413939
Killed by Briar with I

In [86]:
# Extract relevant data from JSON
framesData = data['info']['frames']
participantsData = [frame['participantFrames'] for frame in framesData]

# Define participant IDs
participantIds = ['1', '2']

# Display the extracted parameter values
for participantId in participantIds:
    print(f"Participant {participantId} Parameters:")
    for idx, frameData in enumerate(participantsData):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)
        
        print(f"Frame {idx + 1}:")
        print(f"  Total Damage Done: {totalDamageDone}")
        print(f"  Total Damage Done To Champions: {totalDamageDoneToChampions}")
        print(f"  Total Damage Taken: {totalDamageTaken}")


Participant 1 Parameters:
Frame 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 2:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 3:
  Total Damage Done: 544
  Total Damage Done To Champions: 105
  Total Damage Taken: 291
Frame 4:
  Total Damage Done: 4071
  Total Damage Done To Champions: 1053
  Total Damage Taken: 774
Frame 5:
  Total Damage Done: 5753
  Total Damage Done To Champions: 1195
  Total Damage Taken: 1015
Frame 6:
  Total Damage Done: 11940
  Total Damage Done To Champions: 2301
  Total Damage Taken: 1937
Frame 7:
  Total Damage Done: 12122
  Total Damage Done To Champions: 2301
  Total Damage Taken: 1968
Frame 8:
  Total Damage Done: 20977
  Total Damage Done To Champions: 3833
  Total Damage Taken: 3208
Frame 9:
  Total Damage Done: 23368
  Total Damage Done To Champions: 4180
  Total Damage Taken: 3677
Frame 10:
  Total Damage Done: 33392
  Total Damage Done To Champions: 5733
  Total

In [87]:
def getTimestampOfChampionKillWithId(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')
                victimNames = [victim.get('name') for victim in event.get('victimDamageDealt', [])]
                killerNames = [killer.get('name') for killer in event.get('victimDamageReceived', [])]

                if victimNames:  # Check if there are victim names
                    victimName = victimNames[0]
                    print(f"{victimName} was killed with ID: {victimId}")
                else:
                    print(f"{victimId}")

                if killerNames:  # Check if there are killers
                    # Use the last killer as the one who landed the final hit
                    lastKiller = killerNames[-1]
                    print(f"Killed by {lastKiller} with ID: {killerId}")
                    print(f"Timestamp of CHAMPION_KILL: {timestamp}\n")


getTimestampOfChampionKillWithId(data)


Sion was killed with ID: 6
Killed by Volibear with ID: 1
Timestamp of CHAMPION_KILL: 134226

Brand was killed with ID: 5
Killed by Jhin with ID: 9
Timestamp of CHAMPION_KILL: 171327

Kaisa was killed with ID: 4
Killed by Jhin with ID: 9
Timestamp of CHAMPION_KILL: 187450

Veigar was killed with ID: 8
Killed by Hwei with ID: 3
Timestamp of CHAMPION_KILL: 192190

Hwei was killed with ID: 3
Killed by Kayn with ID: 7
Timestamp of CHAMPION_KILL: 209499

Kayn was killed with ID: 7
Killed by Volibear with ID: 2
Timestamp of CHAMPION_KILL: 255814

Sion was killed with ID: 6
Killed by Briar with ID: 1
Timestamp of CHAMPION_KILL: 260683

Nautilus was killed with ID: 10
Killed by SRU_OrderMinionRanged with ID: 4
Timestamp of CHAMPION_KILL: 303299

Kayn was killed with ID: 7
Killed by Briar with ID: 2
Timestamp of CHAMPION_KILL: 352366

Sion was killed with ID: 6
Killed by SRU_OrderMinionRanged with ID: 1
Timestamp of CHAMPION_KILL: 389728

Veigar was killed with ID: 8
Killed by Hwei with ID: 2
Ti

In [88]:
def getParticipantIdByKill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events',[]):
            eventType = event.get('type')
            if eventType == 'CHAMPION_KILL':
                killerId = event.get('killerId')
                print(f"Participant id: {killerId}") 

#getParticipantIdByKill(data)

In [89]:
# Extract relevant data from JSON
framesData = data['info']['frames']
participantsData = [frame['participantFrames'] for frame in framesData]

# Define participant ID and time frames to subtract
participantIdToSubtract = '1'
startFrame = 3  # Adjust as needed
endFrame = 4    # Adjust as needed

# Initialize variables to store total damage done to champions
totalDamageToChampionsStart = 0
totalDamageToChampionsEnd = 0

# Display the extracted parameter values and calculate total damage done to champions
for idx, frameData in enumerate(participantsData):
    damageStats = frameData.get(participantIdToSubtract, {}).get('damageStats', {})
    totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

    if idx + 1 == startFrame:
        totalDamageToChampionsStart = totalDamageDoneToChampions
    elif idx + 1 == endFrame:
        totalDamageToChampionsEnd = totalDamageDoneToChampions

# Print the total damage done to champions only for the specified frames
print(f"PRE")
print(f"Frame {startFrame}:")
print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
print(f"POST")
print(f"Frame {endFrame}:")
print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")

# Calculate and print the difference
difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
print(f"\nTotal Damage Done To Champions Difference: {difference}")


PRE
Frame 3:
  Total Damage Done To Champions: 105
POST
Frame 4:
  Total Damage Done To Champions: 1053

Total Damage Done To Champions Difference: 948


In [90]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):
    # Extract relevant data from pre-loaded JSON
    framesData = data['info']['frames']
    participantsData = [frame['participantFrames'] for frame in framesData]

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantsData):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions

    # Print the total damage done to champions only for the specified frames
    print(f"PRE")
    print(f"Frame {startFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"POST")
    print(f"Frame {endFrame}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")

    # Calculate and print the difference
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    print(f"\nTotal Damage Done To Champions Difference: {difference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '1', 7, 8)


PRE
Frame 7:
  Total Damage Done To Champions: 2301
POST
Frame 8:
  Total Damage Done To Champions: 3833

Total Damage Done To Champions Difference: 1532


In [91]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):
    # Extract relevant data from pre-loaded JSON
    framesData = data['info']['frames']
    participantsData = [frame['participantFrames'] for frame in framesData]

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done to champions
    totalDamageToChampionsStart = 0
    totalDamageToChampionsEnd = 0
    totalDamageToChampionsExtraStart = 0
    totalDamageToChampionsExtraEnd = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frameData in enumerate(participantsData):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == startFrame:
            totalDamageToChampionsStart = totalDamageDoneToChampions
            totalDamageToChampionsExtraStart = totalDamageDoneToChampionsExtra
        elif idx + 1 == endFrame:
            totalDamageToChampionsEnd = totalDamageDoneToChampions
            totalDamageToChampionsExtraEnd = totalDamageDoneToChampionsExtra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Frame {startFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsStart}")
    print(f"Frame {startFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraStart}")
    
    print(f"POST")
    print(f"Frame {endFrame} - Participant {participantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsEnd}")
    print(f"Frame {endFrame} - Extra Participant {extraParticipantId}:")
    print(f"  Total Damage Done To Champions: {totalDamageToChampionsExtraEnd}")

    # Calculate and print the differences
    difference = totalDamageToChampionsEnd - totalDamageToChampionsStart
    extraDifference = totalDamageToChampionsExtraEnd - totalDamageToChampionsExtraStart
    print(f"\nTotal Damage Done To Champions Difference (Participant {participantId}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extraParticipantId}): {extraDifference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '6', 4, 5)


PRE
Frame 4 - Participant 6:
  Total Damage Done To Champions: 420
Frame 4 - Extra Participant 1:
  Total Damage Done To Champions: 1053
POST
Frame 5 - Participant 6:
  Total Damage Done To Champions: 605
Frame 5 - Extra Participant 1:
  Total Damage Done To Champions: 1195

Total Damage Done To Champions Difference (Participant 6): 185
Total Damage Done To Champions Difference (Extra Participant 1): 142


In [92]:
def calculateDamageDifference(data, participantId, startFrame, endFrame):
    # Extract relevant data from pre-loaded JSON
    framesData = data['info']['frames']
    participantsData = [frame['participantFrames'] for frame in framesData]

    # Determine the extra participantId based on the given participantId
    extraParticipantId = str((int(participantId) + 5) % 10) if int(participantId) < 6 else str((int(participantId) - 5) % 10)

    # Initialize variables to store total damage done parameters
    totalDamageDoneStart = 0
    totalDamageDoneEnd = 0
    totalDamageDoneExtraStart = 0
    totalDamageDoneExtraEnd = 0

    totalDamageDoneToChampionsStart = 0
    totalDamageDoneToChampionsEnd = 0
    totalDamageDoneToChampionsExtraStart = 0
    totalDamageDoneToChampionsExtraEnd = 0

    totalDamageTakenStart = 0
    totalDamageTakenEnd = 0
    totalDamageTakenExtraStart = 0
    totalDamageTakenExtraEnd = 0
    
    totalDamageTakenCompared = 0
    totalDamageDoneCompared = 0
    totalDamageDoneToChampionsCompared = 0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frameData in enumerate(participantsData):
        damageStats = frameData.get(participantId, {}).get('damageStats', {})
        totalDamageDone = damageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampions = damageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTaken = damageStats.get('totalDamageTaken', 0)

        extraDamageStats = frameData.get(extraParticipantId, {}).get('damageStats', {})
        totalDamageDoneExtra = extraDamageStats.get('totalDamageDone', 0)
        totalDamageDoneToChampionsExtra = extraDamageStats.get('totalDamageDoneToChampions', 0)
        totalDamageTakenExtra = extraDamageStats.get('totalDamageTaken', 0)

        if idx + 1 == startFrame:
            totalDamageDoneStart = totalDamageDone
            totalDamageDoneToChampionsStart = totalDamageDoneToChampions
            totalDamageTakenStart = totalDamageTaken
            totalDamageDoneExtraStart = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraStart = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraStart = totalDamageTakenExtra
        elif idx + 1 <= endFrame:
            totalDamageDoneEnd = totalDamageDone
            totalDamageDoneToChampionsEnd = totalDamageDoneToChampions
            totalDamageTakenEnd = totalDamageTaken
            totalDamageDoneExtraEnd = totalDamageDoneExtra
            totalDamageDoneToChampionsExtraEnd = totalDamageDoneToChampionsExtra
            totalDamageTakenExtraEnd = totalDamageTakenExtra
    
    # Print the pre and post data for the first participant
    print(f"Participant {participantId}:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenEnd}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner:")
    print(f"  Frame {startFrame} - Total Damage Done: {totalDamageDoneExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done: {totalDamageDoneExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Done To Champions: {totalDamageDoneToChampionsExtraEnd}")
    print(f"  Frame {startFrame} - Total Damage Taken: {totalDamageTakenExtraStart}")
    print(f"  Frame {endFrame} - Total Damage Taken: {totalDamageTakenExtraEnd}")

    # Calculate and print the differences
    damageDoneDifference = totalDamageDoneEnd - totalDamageDoneStart
    extraDamageDoneDifference = totalDamageDoneExtraEnd - totalDamageDoneExtraStart
    damageDoneToChampionsDifference = totalDamageDoneToChampionsEnd - totalDamageDoneToChampionsStart
    extraDamageDoneToChampionsDifference = totalDamageDoneToChampionsExtraEnd - totalDamageDoneToChampionsExtraStart
    damageTakenDifference = totalDamageTakenEnd - totalDamageTakenStart
    extraDamageTakenDifference = totalDamageTakenExtraEnd - totalDamageTakenExtraStart

    print(f"\nTotal Damage Done Difference (Participant {participantId}): {damageDoneDifference}")
    print(f"Total Damage Done Difference (Enemy Laner): {extraDamageDoneDifference}")
    print(f"Total Damage Done To Champions Difference (Participant {participantId}): {damageDoneToChampionsDifference}")
    print(f"Total Damage Done To Champions Difference (Enemy Laner): {extraDamageDoneToChampionsDifference}")
    print(f"Total Damage Taken Difference (Participant {participantId}): {damageTakenDifference}")
    print(f"Total Damage Taken Difference (Enemy Laner): {extraDamageTakenDifference}")
    
    # Calculate differences between participant and enemy
    totalDamageTakenCompared = damageTakenDifference - extraDamageTakenDifference
    totalDamageDoneCompared = damageDoneDifference - extraDamageDoneDifference
    totalDamageDoneToChampionsCompared = damageDoneToChampionsDifference - extraDamageDoneToChampionsDifference
    
    if totalDamageTakenCompared >= 0:
        print(f"\nParticipant {participantId} took {totalDamageTakenCompared} more damage than enemy laner in given timeframe")
    else:
        print(f"\nParticipant {participantId} took {abs(totalDamageTakenCompared)} less damage than enemy laner in given timeframe")

    if totalDamageDoneToChampionsCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneToChampionsCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneToChampionsCompared)}")

    if totalDamageDoneCompared >= 0:
        print(f"\nParticipant {participantId} beat enemy laner in given timeframe with total damage done difference of: {totalDamageDoneCompared}")
    else:
        print(f"\nParticipant {participantId} was beat by enemy laner in given timeframe with total damage done difference of: {abs(totalDamageDoneCompared)}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculateDamageDifference(data, '6', 4, 6)


Participant 6:
  Frame 4 - Total Damage Done: 3198
  Frame 6 - Total Damage Done: 5736
  Frame 4 - Total Damage Done To Champions: 420
  Frame 6 - Total Damage Done To Champions: 870
  Frame 4 - Total Damage Taken: 1444
  Frame 6 - Total Damage Taken: 3223

Enemy Laner:
  Frame 4 - Total Damage Done: 4071
  Frame 6 - Total Damage Done: 11940
  Frame 4 - Total Damage Done To Champions: 1053
  Frame 6 - Total Damage Done To Champions: 2301
  Frame 4 - Total Damage Taken: 774
  Frame 6 - Total Damage Taken: 1937

Total Damage Done Difference (Participant 6): 2538
Total Damage Done Difference (Enemy Laner): 7869
Total Damage Done To Champions Difference (Participant 6): 450
Total Damage Done To Champions Difference (Enemy Laner): 1248
Total Damage Taken Difference (Participant 6): 1779
Total Damage Taken Difference (Enemy Laner): 1163

Participant 6 took 616 more damage than enemy laner in given timeframe

Participant 6 was beat by enemy laner in given timeframe with total damage done diff